In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3
import glob
import subprocess

In [3]:
# First figure out annuitised costs for the capital costs 

capcost = [1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000]
cost = []

for i in range(len(capcost)):
    cost.append(capcost[i]/(sum(1/((1+0.05)**x) for x in range(1,11))))

    
for h in range(len(cost)):
    cost[h] = round(cost[h] / 3)
    
    
print(cost)


[43, 65, 86, 108, 130, 151, 173, 194, 216, 237, 259, 281, 302, 324, 345, 367, 389, 410, 432]


In [6]:
with open("EVCS_15_A.gms","r") as f:
    list_of_lines = f.readlines()
    
list_of_lines[125]

'\n'

In [7]:
for i in range(len(cost)):
    
    with open("EVCS_15_A.gms","r") as f:
        list_of_lines = f.readlines()
    
    list_of_lines[1] = "scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /" + str(cost[i]) + "/\n"
    
    list_of_lines[125] = "var_ESS_size.fx = 142.964;\n"
    
    print(list_of_lines[1])
    
    with open("EVCS_15_A.gms", "w") as f:
        f.writelines(list_of_lines)
        
    subprocess.run(["gams", "EVCS_15_A.gms", "logOption", "2"])
    
    subprocess.run(["move", "EVCS_15_A_results.db", "EVCS_15_A2_results" + str(cost[i]) + ".db"], shell = True)
    
    print(cost[i])

scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /43/

43
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /65/

65
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /86/

86
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /108/

108
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /130/

130
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /151/

151
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /173/

173
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /194/

194
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /216/

216
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /237/

237
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /259/

259
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /281/

281
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /302/

302
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /324/

324
scalar battery_cost 'Cost of Li-Oin battery NOK/kWh' /345/

345
scalar battery_cost 'Cost of Li-Oin battery NO

In [4]:
#Extract result data from all result databases and create dataframes for each variable

grid_p = pd.DataFrame()
grid_ex_p = pd.DataFrame()
grid_tot_p = pd.DataFrame()

soc_pen = pd.DataFrame()

grid_e = pd.DataFrame()
grid_ex_e = pd.DataFrame()

ESS_lvl = pd.DataFrame()
ESS_p = pd.DataFrame()

sclrs = pd.DataFrame()

for filename in glob.glob('EVCS_15_A2_results*.db'):
    con = sqlite3.connect(filename)
    
    grid_power = pd.read_sql_query("SELECT * FROM var_grid_power", con)
    grid_extra_power = pd.read_sql_query("SELECT * FROM var_grid_ex_power", con)
    grid_total_power = pd.read_sql_query("SELECT * FROM var_grid_total_power", con)
    
    grid_energy = pd.read_sql_query("SELECT * FROM var_grid_energy", con)
    grid_extra_energy = pd.read_sql_query("SELECT * FROM var_grid_ex_energy", con)
    
    ESS_level = pd.read_sql_query('SELECT * FROM var_ESS_level', con)
    ESS_power = pd.read_sql_query('SELECT * FROM var_ESS_power', con)
    
    scalars =  pd.read_sql_query('SELECT * FROM scalarvariables', con)
    
    soc_penalty = pd.read_sql_query('SELECT * FROM var_SOC_pen', con)
    
    con.close()
    
    grid_p[filename[18:-3]] = grid_power['level'].copy()
    grid_ex_p[filename[18:-3]] = grid_extra_power['level'].copy()
    grid_tot_p[filename[18:-3]] = grid_total_power['level'].copy()
    
    grid_e[filename[18:-3]] = grid_energy['level'].copy()
    grid_ex_e[filename[18:-3]] = grid_extra_energy['level'].copy() 
    
    ESS_lvl[filename[18:-3]] = ESS_level['level'].copy()
    ESS_p[filename[18:-3]] = ESS_power['level'].copy()  
    
    sclrs[filename[18:-3]] = scalars['level'].copy()
    
    soc_pen[filename[18:-3]] = soc_penalty['level'].copy()



In [5]:
sclrs = sclrs.set_index(scalars['name'])
sclrs1 = sclrs.T
sclrs1['var_ESS_size'] = sclrs1['var_ESS_size'] * 1.25
sclrs1.index = sclrs1.index.astype(int)
sclrs1 = sclrs1.sort_index(ascending = True)
sclrs1['Battery Investment Costs'] = np.array(capcost)
sclrs1

name,var_ess_rated_power,var_grid_maxpower1,var_grid_maxpower2,var_grid_maxpower3,var_grid_maxpower4,var_ESS_cost,var_electricity_cost,var_grid_tariff_winter,var_energy_tax,months,var_system_cost,var_ESS_size,Battery Investment Costs
43,178.705,217.855,262.98878,177.57504,220.765,6147.452,141080.248603,69593.15502,3602.265477,4.283482,224930.466144,178.705,1000
65,178.705,217.855,262.98878,177.57504,220.765,9292.660,141080.248603,69593.15502,3602.265477,4.283482,228075.674144,178.705,1500
86,178.705,217.855,262.98878,177.57504,220.765,12294.904,141080.248603,69593.15502,3602.265477,4.283482,231077.918144,178.705,2000
108,178.705,217.855,262.98878,177.57504,220.765,15440.112,141080.248603,69593.15502,3602.265477,4.283482,234223.126144,178.705,2500
130,178.705,217.855,262.98878,177.57504,220.765,18585.320,141080.248603,69593.15502,3602.265477,4.283482,237368.334144,178.705,3000
151,178.705,217.855,262.98878,177.57504,220.765,21587.564,141080.248603,69593.15502,3602.265477,4.283482,240370.578144,178.705,3500
173,178.705,217.855,262.98878,177.57504,220.765,24732.772,141080.248603,69593.15502,3602.265477,4.283482,243515.786144,178.705,4000
194,0.000,0.000,0.00000,0.00000,0.000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,178.705,4500
216,0.000,0.000,0.00000,0.00000,0.000,0.000,0.000000,0.00000,0.000000,0.000000,0.000000,178.705,5000
237,178.705,217.855,262.98878,177.57504,220.765,33882.468,141080.248603,69593.15502,3602.265477,4.283482,252665.482144,178.705,5500
